In [ ]:
!pip install -U datasets evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [ ]:
import multiprocessing
from multiprocessing import Pool

import torch
from tqdm import tqdm
from evaluate import load
from datasets import load_dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

def run_eval(dataset):
    print("Evaluating", dataset)

    if torch.cuda.is_available():
        device = 0                 # GPU‑0
        torch_dtype = torch.float16
    else:
        device = -1                # CPU
        torch_dtype = torch.float32

    pipe = pipeline(
        "automatic-speech-recognition",
        model="rbcurzon/whisper-small-ph",
        device=device,
        torch_dtype=torch_dtype,
    )

    all_predictions = [
        pred["text"]
        for pred in tqdm(
            pipe(
                KeyDataset(dataset, "audio"),
                return_timestamps=True,
                generate_kwargs={
                    "task": "transcribe",
                    "language": "tagalog",
                    "max_new_tokens": 128,
                },
            ),
            total=len(dataset),
        )
    ]

    wer_metric = load("wer")
    wer = 100 * wer_metric.compute(
        references=dataset["transcription"], predictions=all_predictions
    )
    return f"{dataset}: {wer:.2f}% WER"

In [ ]:
from multiprocessing import Pool
from datasets import load_dataset
import multiprocessing
from functools import partial

def load_hf_datasets():
  ds_list = []
  subsets = ["bik", "ceb", "hil", "ilo", "mrw", "pag", "tgl", "war", "pam", "bisaya"]

  with Pool(multiprocessing.cpu_count()) as pool:
    # Use partial to pre-apply the 'split' argument and the dataset path to load_dataset
    load_dataset_with_split = partial(load_dataset, "rbcurzon/ph_dialect_asr", split="test")
    for subset in pool.imap_unordered(load_dataset_with_split, subsets):
      print(f"Loading dataset for subset: {subset}")
      ds_list.append(subset)

  return ds_list

In [ ]:
ds_list = load_hf_datasets()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/76.1M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.3M [00:00<?, ?B/s]

...

test-00002-of-00004.parquet:   0%|          | 0.00/542M [00:00<?, ?B/s]

test-00003-of-00004.parquet:   0%|          | 0.00/781M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9938 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2705 [00:00<?, ? examples/s]

Loading dataset for subset: Dataset({
    features: ['audio', 'transcription', 'raw', 'id', 'num_samples', 'path', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 2705
})


In [ ]:
results = []

for ds in ds_list:
  results.append(run_eval(ds))

Evaluating Dataset({
    features: ['audio', 'transcription', 'raw'],
    num_rows: 110
})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

...

Device set to use cuda:0
100%|██████████| 964/964 [22:11<00:00,  1.38s/it]


Evaluating Dataset({
    features: ['audio', 'transcription'],
    num_rows: 219
})


Device set to use cuda:0
100%|██████████| 219/219 [02:34<00:00,  1.42it/s]


Evaluating Dataset({
    features: ['audio', 'transcription', 'raw', 'id', 'num_samples', 'path', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 2705
})


Device set to use cuda:0
100%|██████████| 2705/2705 [52:11<00:00,  1.16s/it]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

# Example data
subsets = ["bik", "ceb", "hil", "ilo", "mrw", "pag", "tgl", "war", "pam", "bisaya"]
y_pos = np.arange(len(subsets))

small_model = results

# medium_model = [16.030283080974325, 21.57816005983545, 15.73165947430365, 17.93032786885246]

for model, scores in zip(['small'], [small_model]):
    ax.barh(y_pos, scores, height=0.4, color='steelblue', label=model)

for i, v in enumerate(scores):
    ax.text(v + 0.3, i, f"{v:.2f}", va='center', fontsize=10)

ax.set_yticks(y_pos)
ax.set_yticklabels(subsets, fontsize=10)
ax.invert_yaxis()

ax.set_xlabel('Word Error Rate (%)', fontsize=12)
ax.set_title('WER by Dialect – Small Model', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig("small_model_wer.png", dpi=300)
plt.show()
